# Working with Mnemonics

## THIS PROGRAM COMES WITH NO GUARANTEES WHATSOEVER!
## MAKE SURE YOU ARE USING THIS PROGRAM IN A SECURE MACHINE!

Fill in the mnemonic, password, destination, segwit and coin_class below and run

In [ ]:
from hd import HDPrivateKey
from helper import decode_base58, h160_to_p2pkh_address, h160_to_p2sh_address
from tx import Tx, BCHTx, BTGTx, SBTCTx, BCDTx, BTXTx, B2XTx, BCXTx, BTFTx, BTWTx, BTVTx, BTCPTx, LBTCTx, BCITx, BTPTx

mnemonic = '' # your trezor/ledger seed here
password = b'' # usually this is blank, but if you used the 25th word, then put it here
destination = '' # should be the address you are splitting the coins to
segwit = True # did you use segwit addresses or not?
coin_class = BCHTx # change this to whichever coin you want

if segwit:
    path = b"m/49'/0'"
    prefix = coin_class.p2sh_prefixes[0]
else:
    path = b"m/44'/0'"
    prefix = coin_class.p2pkh_prefixes[0]

# sanity check
if not decode_base58(destination)[:len(prefix)] == prefix:
    raise RuntimeError('Please check that the destination is an address for the coin')

# get the corresponding HD key
hd_priv_key = HDPrivateKey.from_mnemonic(mnemonic, password, path)

# get all the active paths that have ever received a coin
mnemonic_paths = hd_priv_key.get_active_paths(
    account_gap=1,
    gap_limit=20,
    segwit=segwit,
)

# get all the utxos associated with every active address for this coin
path_utxos = []
for this_path, addr in mnemonic_paths:
    h160 = decode_base58(addr)[1:]
    if segwit:
        addr = h160_to_p2sh_address(h160, prefix=prefix)
    else:
        addr = h160_to_p2pkh_address(h160, prefix=prefix)
    addr_utxos = coin_class.fetch_address_utxos(addr)
    if addr_utxos:
        path_utxos.append((this_path, addr_utxos))
        
# create a transaction where all the inputs are the unspent outputs to a single destination
tx = HDPrivateKey.spend_all_tx_from_mnemonic(
    coin_class=coin_class,
    mnemonic=mnemonic,
    password=password,
    path=path,
    destination=destination,
    fee=coin_class.fee,
    path_utxos=path_utxos,
    segwit=segwit,
)

# print the raw transaction
print(tx.serialize().hex())

# uncomment if you want to auto-send
#if tx.insight:
#    tx.send_insight()

# Working with WIFs

## THIS PROGRAM COMES WITH NO GUARANTEES WHATSOEVER!
## MAKE SURE YOU ARE USING THIS PROGRAM IN A SECURE MACHINE!

Fill in the wifs, destination segwit and coin_class below and run

In [ ]:
from helper import decode_base58, h160_to_p2pkh_address, h160_to_p2sh_address, hash160
from tx import Tx, BCHTx, BTGTx, SBTCTx, BCDTx, BTXTx, B2XTx, BCXTx, BTFTx, BTWTx, BTVTx, BTCPTx, LBTCTx, BCITx, BTPTx
from ecc import PrivateKey

wifs = [''] # WIF format strings here
destination = '' # should be the address you are splitting the coins to
segwit = False # did you use segwit addresses or not?
coin_class = BCHTx # change this to whichever coin you want

# get all the hash160s
h160s = [PrivateKey.parse(wif).h160() for wif in wifs]

# convert to addresses
if segwit:
    prefix = coin_class.p2sh_prefixes[0]
    addrs = [h160_to_p2sh_address(hash160(b'\x16\x00\x14' + h160), prefix=prefix) for h160 in h160s]
else:
    prefix = coin_class.p2pkh_prefixes[0]
    addrs = [h160_to_p2pkh_address(h160, prefix=prefix) for h160 in h160s]

# sanity check
if not decode_base58(destination)[:len(prefix)] == prefix:
    raise RuntimeError('Please check that the destination is an address for the coin')

# get all the utxos associated with these addresses for this coin
utxos = coin_class.get_all_utxos(addrs)
if not utxos:
    raise RuntimeError('No utxos for these wifs')
    
# create a transaction spending everything to the destination
tx = coin_class.spend_tx(wifs, utxos, destination, coin_class.fee, segwit=segwit)

# print the raw transaction
print(tx.serialize().hex())

# uncomment if you want to auto-send
#if tx.insight:
#    tx.send_insight()